In [16]:

import numpy as np
import json
import cv2
import json
import os
import random

from src.HydraPose import HydraPose
hy = HydraPose()

import sys
sys.path.insert(0, '/home/guisoares/soares_repo/MVOR/lib/')

from visualize_groundtruth import create_index


Starting device with CPU...


In [18]:
GT_ANNO_PATH = os.path.join(os.path.expanduser('~'), "soares_repo", "MVOR","annotations/camma_mvor_2018.json")
GT_IMGS_PATH = '/media/guisoares/guisoares-ext-hdd/Datasets/camma_mvor_dataset/'

# load the ground truth annotations
camma_mvor_gt = json.load(open(GT_ANNO_PATH))
anno_2d, anno_3d, mv_paths, imid_to_path = create_index(camma_mvor_gt)

creating index for 2D annotations
done
creating index for 3D annotations
Index creation done


In [6]:
# imid_to_path.keys()
imid_to_path[10020000017]

'day1/cam2/color/000017.png'

Some functions to make the evaluation process

In [ ]:

def hydrapose(img_path):
    img = cv2.imread(img_path)
    persons = hy.estimate3DPose()
    return persons

def getPersonMediumDist(person1, person2):
    dists = []
    for kpt1, kpt2 in zip(person1, person2):
        dist = np.sqrt(np.sum(np.power(kpt1 - kpt2, 2)))
        dists.append(dist)
    
    dists = np.array(dists)
    mean = np.mean(dists)
    return mean, dists

def getMinimalDist(persons_est, persons_annon):

    # Store
    min_index_list = []
    min_error_per_joint_list = []
    min_mean_error_per_joint_list = []
    # Get the annotation index that results the minimal mean error
    for person_est in persons_est:
        mean_error_list = []
        error_per_joint_list = []
        for person_annon in persons_annon:
            mean_error, error_per_joint = getMinimalDist(person_est, person_annon)
            mean_error_list.append(mean_error)
            error_per_joint_list.append(error_per_joint)
        # Get the index of the minimum error and insert on a list that contains it 
        min_index = np.argmin(mean_error_list)
        min_index_list.append(min_index)
        min_error_per_joint_list.append(error_per_joint_list[min_index])
        min_mean_error_per_joint_list.append(mean_error_list[min_index])
    
    # Certifie that index list has diferent elements, meaning that no estimation has the same annotation
    _, count = np.unique(min_index_list, return_counts = True)
    for c in count:
        if c != 1:
            raise AssertionError("The mapped index list has duplicated elements")

    # Remap annotation to
    persons_annon_remapped = persons_annon[min_index_list]

    return persons_est, persons_annon_remapped, min_mean_error_per_joint_list, min_error_per_joint_list

In [19]:
# Read a random multi-view image
imid_3d = random.choice(list(mv_paths.keys()))
imids_2d = [int(m) for m in imid_3d.split("_")]
imgs = [cv2.imread(os.path.join(GT_IMGS_PATH, imid_to_path[_p])) for _p in imids_2d]

NameError: name 'DATASET_IMG_PATH' is not defined